In [36]:
from PIL import Image, ImageDraw, ImageFont
import io

In [37]:
def cmap(value):
    pixval = value * 255
    low = 64
    high = 240
    factor = (255 - low - (255-high)) / 255
    return int(low + pixval * factor)

def draw_sequence(y, u=12):
    seq_len = y.size(0)
    seq_width = y.size(1)
    inset = u // 8
    pad = u // 2
    width = seq_len * u + 2 * pad
    height = seq_width * u + 2 * pad
    im = Image.new('L', (width, height))
    draw = ImageDraw.ImageDraw(im)
    draw.rectangle([0, 0, width, height], fill=250)
    for i in range(seq_len):
        for j in range(seq_width):
            val = 1 - y[i, j].data[0]
            draw.rectangle([pad + i*u + inset,
                            pad + j*u + inset,
                            pad + (i+1)*u - inset,
                            pad + (j+1)*u - inset], fill=cmap(val))

    return im

def im_to_png_bytes(im):
    png = io.BytesIO()
    im.save(png, 'PNG')
    # return bytes(png.getbuffer())

def im_vconcat(im1, im2, pad=8):
    assert im1.size == im2.size
    w, h = im1.size

    width = w
    height = h * 2 + pad

    im = Image.new('L', (width, height), color=255)
    im.paste(im1, (0, 0))
    im.paste(im2, (0, h+pad))
    return im

In [38]:
def make_eval_plot(y, y_out, u=12):
    im_y = draw_sequence(y, u)
    im_y_out = draw_sequence(y_out, u)
    im = im_vconcat(im_y, im_y_out, u//2)
    
    w, h = im.size
    pad_w = u * 7
    im2 = Image.new('L', (w+pad_w, h), color=255)
    im2.paste(im, (pad_w, 0))
    return im2

In [39]:
%run -i evaluate.py
output = result['output']
target = result['target']

Loss: 0.01	Error in bits per sequence: 0.00


/Users/pragyasingh/miniconda3/envs/ntm/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'ntm.ntm.NTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/pragyasingh/miniconda3/envs/ntm/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'ntm.modules.controller.NTMController' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/pragyasingh/miniconda3/envs/ntm/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'ntm.modules.memory.NTMMemory' has changed. you can retrieve the original 

In [40]:
# in the images folder, the upper plot represents target sequence and lower represents output of ntm 
im = make_eval_plot(target, output, u=12)
im.save('images/prioritysort.png')
# for repeat copy the bit error is mostly due to it's inability to correctly predict the end marker

evaluate.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  '''
